# Motivation : 
Nous sommes un groupe d'élèves ingénieurs du cercle MIS de l'INP-HB. Notre projet consiste en la détection d'anomalies en cyber sécurité via une plateforme qui surveille les activités sur les réseaux informatiques et utilise des algorithmes d'apprentissage automatique pour détecter les activités suspectes. Nous avons entraîné notre modèle à identifier les schémas d'activité normaux, afin de pouvoir détecter rapidement les activités anormales qui pourraient signaler une attaque. Nous planifions d'alerter les analystes de sécurité en cas de détection d'anomalies

# Algorithme implémenté :
- Détection d'anomalies basée sur les clusters (K-mean)
- Répartition des données en catégories puis Enveloppe Gaussienne/Elliptique sur chaque catégorie séparément
- Chaîne de Markov
- Forêt d'isolement
- Une classe SVM
- RNN (comparaison entre prédiction et réalité)

In [ ]:
# libraries
#%matplotlib notebook

import pandas as pd
import numpy as np

import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
#from pyemma import msm # not available on Kaggle Kernel
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

In [ ]:
# some function for later

# return Series of distance between each point and his distance with the closest centroid
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.set_value(i, np.linalg.norm(Xa-Xb))
    return distance

# train markov model to get transition matrix
def getTransitionMatrix (df):
	df = np.array(df)
	model = msm.estimate_markov_model(df, 1)
	return model.transition_matrix

def markovAnomaly(df, windows_size, threshold):
    transition_matrix = getTransitionMatrix(df)
    real_threshold = threshold**windows_size
    df_anomaly = []
    for j in range(0, len(df)):
        if (j < windows_size):
            df_anomaly.append(0)
        else:
            sequence = df[j-windows_size:j]
            sequence = sequence.reset_index(drop=True)
            df_anomaly.append(anomalyElement(sequence, real_threshold, transition_matrix))
    return df_anomaly

# 1 Données
## 1.1 Extraire les données
L'ensemble de données provient de https://www.kaggle.com/boltzmannbrain/nab
Dans realKnownCause/ambient_temperature_system_failure.csv

In [ ]:
df = pd.read_csv("../input/realKnownCause/realKnownCause/ambient_temperature_system_failure.csv")

## 1.2 Understand data

In [ ]:
print(df.info())

In [ ]:
# vérifier le format et la fréquence de l'horodatage
print(df['timestamp'].head(10))

In [ ]:
# vérifier la température moyenne
print(df['value'].mean())

In [ ]:
# modifier le type de colonne d'horodatage pour le traçage
df['timestamp'] = pd.to_datetime(df['timestamp'])
# changer les degrés Fahrenheit en °C (température moyenne = 71 -> Fahrenheit)
df['value'] = (df['value'] - 32) * 5/9
# tracer les données
df.plot(x='timestamp', y='value')

## 1.3 Ingénierie des fonctionnalités
Extraire quelques fonctionnalités utiles

In [ ]:
# les heures et si c'est la nuit ou le jour (7:00-22:00)
df['hours'] = df['timestamp'].dt.hour
df['daylight'] = ((df['hours'] >= 7) & (df['hours'] <= 22)).astype(int)

In [ ]:
# le jour de la semaine (lundi=0, dimanche=6) et s'il s'agit d'un jour de week-end ou d'un jour de semaine.
df['DayOfTheWeek'] = df['timestamp'].dt.dayofweek
df['WeekDay'] = (df['DayOfTheWeek'] < 5).astype(int)
# Une estimation de la population anormale du jeu de données (nécessaire pour plusieurs algorithmes)
outliers_fraction = 0.01

In [ ]:
# temps avec int pour tracer facilement
df['time_epoch'] = (df['timestamp'].astype(np.int64)/100000000000).astype(np.int64)

In [ ]:
# création de 4 catégories distinctes qui semblent utiles (week end/jour semaine & nuit/jour)
df['categories'] = df['WeekDay']*2 + df['daylight']

a = df.loc[df['categories'] == 0, 'value']
b = df.loc[df['categories'] == 1, 'value']
c = df.loc[df['categories'] == 2, 'value']
d = df.loc[df['categories'] == 3, 'value']

fig, ax = plt.subplots()
a_heights, a_bins = np.histogram(a)
b_heights, b_bins = np.histogram(b, bins=a_bins)
c_heights, c_bins = np.histogram(c, bins=a_bins)
d_heights, d_bins = np.histogram(d, bins=a_bins)

width = (a_bins[1] - a_bins[0])/6

ax.bar(a_bins[:-1], a_heights*100/a.count(), width=width, facecolor='blue', label='WeekEndNight')
ax.bar(b_bins[:-1]+width, (b_heights*100/b.count()), width=width, facecolor='green', label ='WeekEndLight')
ax.bar(c_bins[:-1]+width*2, (c_heights*100/c.count()), width=width, facecolor='red', label ='WeekDayNight')
ax.bar(d_bins[:-1]+width*3, (d_heights*100/d.count()), width=width, facecolor='black', label ='WeekDayLight')

plt.legend()
plt.show()

Nous pouvons voir que la température est plus stable pendant la journée du jour ouvrable.
# 2 Modèles
## 2.1 Grappe uniquement
#### À utiliser pour les anomalies collectives (non ordonnées).

Nous regroupons la combinaison habituelle de fonctionnalités. Les points éloignés du cluster sont des points avec une combinaison habituelle de caractéristiques. Nous considérons ces points comme des anomalies.

In [ ]:
# Prenez des fonctionnalités utiles et normalisez-les
data = df[['value', 'hours', 'daylight', 'DayOfTheWeek', 'WeekDay']]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# réduire à 2 fonctionnalités importantes
pca = PCA(n_components=2)
data = pca.fit_transform(data)
# standardiser ces 2 nouvelles fonctionnalités
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)

In [ ]:
# calculer avec un nombre différent de centroïdes pour voir le graphique de perte (méthode du coude)
n_cluster = range(1, 20)
kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]
fig, ax = plt.subplots()
ax.plot(n_cluster, scores)
plt.show()

In [ ]:
# Pas clair pour moi, je choisis arbitrairement 15 centroïdes et ajoute ces données à la trame de données centrale
df['cluster'] = kmeans[14].predict(data)
df['principal_feature1'] = data[0]
df['principal_feature2'] = data[1]
df['cluster'].value_counts()

In [ ]:
# tracer les différents clusters avec les 2 caractéristiques principales
fig, ax = plt.subplots()
colors = {0:'red', 1:'blue', 2:'green', 3:'pink', 4:'black', 5:'orange', 6:'cyan', 7:'yellow', 8:'brown', 9:'purple', 10:'white', 11: 'grey', 12:'lightblue', 13:'lightgreen', 14: 'darkgrey'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["cluster"].apply(lambda x: colors[x]))
plt.show()

In [ ]:
# obtenir la distance entre chaque point et son centre de gravité le plus proche. Les plus grandes distances sont considérées comme des anomalies
distance = getDistanceByPoint(data, kmeans[14])
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
# anomaly21 contient le résultat d'anomalie de la méthode 2.1 Cluster (0 : normal, 1 : anomalie)
df['anomaly21'] = (distance >= threshold).astype(int)

In [ ]:
# visualisation de l'anomalie avec la vue cluster
fig, ax = plt.subplots()
colors = {0:'blue', 1:'red'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["anomaly21"].apply(lambda x: colors[x]))
plt.show()

In [ ]:
# visualisation de l'anomalie dans le temps (viz 1)fig, ax = plt.subplots()

a = df.loc[df['anomaly21'] == 1, ['time_epoch', 'value']] #anomaly

ax.plot(df['time_epoch'], df['value'], color='blue')
ax.scatter(a['time_epoch'],a['value'], color='red')
plt.show()

In [ ]:
# visualisation d'anomalie avec répartition de température (viz 2)
a = df.loc[df['anomaly21'] == 0, 'value']
b = df.loc[df['anomaly21'] == 1, 'value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
plt.legend()
plt.show()

La méthode de cluster détecte la basse température autour de la fin de l'enregistrement comme anormalement basse. Il ne détecte pas la température la plus élevée.
## 2.2 Catégories + Gaussien
#### À utiliser pour les données contextuelles et les anomalies collectives (non ordonnées).
Nous séparerons les données par (ce que nous pensons) des catégories importantes. Ou nous pouvons séparer les données en fonction de différents clusters (méthode 2.3). Ensuite, nous trouvons des valeurs aberrantes (répartition gaussienne, unimodale) par catégories indépendamment.

In [ ]:
# creation of 4 differents data set based on categories defined before
df_class0 = df.loc[df['categories'] == 0, 'value']
df_class1 = df.loc[df['categories'] == 1, 'value']
df_class2 = df.loc[df['categories'] == 2, 'value']
df_class3 = df.loc[df['categories'] == 3, 'value']

In [ ]:
# tracer la répartition de la température par catégories
fig, axs = plt.subplots(2,2)
df_class0.hist(ax=axs[0,0],bins=32)
df_class1.hist(ax=axs[0,1],bins=32)
df_class2.hist(ax=axs[1,0],bins=32)
df_class3.hist(ax=axs[1,1],bins=32)

In [ ]:
# appliquer ellipticEnvelope (distribution gaussienne) à chaque catégorie
envelope =  EllipticEnvelope(contamination = outliers_fraction) 
X_train = df_class0.values.reshape(-1,1)
envelope.fit(X_train)
df_class0 = pd.DataFrame(df_class0)
df_class0['deviation'] = envelope.decision_function(X_train)
df_class0['anomaly'] = envelope.predict(X_train)

envelope =  EllipticEnvelope(contamination = outliers_fraction) 
X_train = df_class1.values.reshape(-1,1)
envelope.fit(X_train)
df_class1 = pd.DataFrame(df_class1)
df_class1['deviation'] = envelope.decision_function(X_train)
df_class1['anomaly'] = envelope.predict(X_train)

envelope =  EllipticEnvelope(contamination = outliers_fraction) 
X_train = df_class2.values.reshape(-1,1)
envelope.fit(X_train)
df_class2 = pd.DataFrame(df_class2)
df_class2['deviation'] = envelope.decision_function(X_train)
df_class2['anomaly'] = envelope.predict(X_train)

envelope =  EllipticEnvelope(contamination = outliers_fraction) 
X_train = df_class3.values.reshape(-1,1)
envelope.fit(X_train)
df_class3 = pd.DataFrame(df_class3)
df_class3['deviation'] = envelope.decision_function(X_train)
df_class3['anomaly'] = envelope.predict(X_train)

In [ ]:
# tracer la répartition de la température par catégories avec des anomalies
a0 = df_class0.loc[df_class0['anomaly'] == 1, 'value']
b0 = df_class0.loc[df_class0['anomaly'] == -1, 'value']

a1 = df_class1.loc[df_class1['anomaly'] == 1, 'value']
b1 = df_class1.loc[df_class1['anomaly'] == -1, 'value']

a2 = df_class2.loc[df_class2['anomaly'] == 1, 'value']
b2 = df_class2.loc[df_class2['anomaly'] == -1, 'value']

a3 = df_class3.loc[df_class3['anomaly'] == 1, 'value']
b3 = df_class3.loc[df_class3['anomaly'] == -1, 'value']

fig, axs = plt.subplots(2,2)
axs[0,0].hist([a0,b0], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
axs[0,1].hist([a1,b1], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
axs[1,0].hist([a2,b2], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
axs[1,1].hist([a3,b3], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
axs[0,0].set_title("WeekEndNight")
axs[0,1].set_title("WeekEndLight")
axs[1,0].set_title("WeekDayNight")
axs[1,1].set_title("WeekDayLight")
plt.legend()
plt.show()

In [ ]:
# ajouter les données à la main
df_class = pd.concat([df_class0, df_class1, df_class2, df_class3])
df['anomaly22'] = df_class['anomaly']
df['anomaly22'] = np.array(df['anomaly22'] == -1).astype(int) 

In [ ]:
# visualisation de l'anomalie dans le temps (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly22'] == 1, ('time_epoch', 'value')] #anomaly

ax.plot(df['time_epoch'], df['value'], color='blue')
ax.scatter(a['time_epoch'],a['value'], color='red')
plt.show()

In [ ]:
# visualisation d'anomalie avec répartition de température (viz 2)
a = df.loc[df['anomaly22'] == 0, 'value']
b = df.loc[df['anomaly22'] == 1, 'value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
plt.legend()
plt.show()

De bonnes détections de valeurs extrêmes et une séparation de contexte ajoutent une certaine précision à la détection.
## 2.3 Cluster+Gaussien
Similaire à la solution 2.2 mais avec un cluster pour séparer les données dans différents groupes.

## 2.4 Chaînes de Markov
#### À utiliser pour les anomalies séquentielles (ordonnées)
Nous devons discrétiser les points de données dans des états définis pour la chaîne de Markov. Nous prendrons juste 'valeur' ​​pour définir l'état de cet exemple et définirons 5 niveaux de valeur (très bas, bas, moyen, haut, très haut)/(VL, L, A, H, VH).
La chaîne de Markov calculera la probabilité de séquence comme (VL, L, L, A, A, L, A). Si la probability is very weak we consider the sequence as an anomaly.

In [ ]:
# définition de l'état différent
x1 = (df['value'] <=18).astype(int)
x2= ((df['value'] > 18) & (df['value']<=21)).astype(int)
x3 = ((df['value'] > 21) & (df['value']<=24)).astype(int)
x4 = ((df['value'] > 24) & (df['value']<=27)).astype(int)
x5 = (df['value'] >27).astype(int)
df_mm = x1 + 2*x2 + 3*x3 + 4*x4 + 5*x5

# obtenir les étiquettes d'anomalie pour notre ensemble de données (évaluation de la séquence de 5 valeurs et anomalie = moins de 20 % de probabilité)
# J'UTILISE pyemma NON DISPONIBLE DANS KAGGLE KERNEL
#df_anomaly = markovAnomaly(df_mm, 5, 0.20)
#df_anomaly = pd.Series(df_anomaly)
#print(df_anomaly.value_counts())

In [ ]:
"""
# add the data to the main 
df['anomaly24'] = df_anomaly

# visualisation of anomaly throughout time (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly24'] == 1, ('time_epoch', 'value')] #anomaly

ax.plot(df['time_epoch'], df['value'], color='blue')
ax.scatter(a['time_epoch'],a['value'], color='red')
plt.show()
"""

In [ ]:
"""
# visualisation of anomaly with temperature repartition (viz 2)
a = df.loc[df['anomaly24'] == 0, 'value']
b = df.loc[df['anomaly24'] == 1, 'value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'])
plt.legend()
plt.show()
"""

Détecter une séquence inhabituelle mais pas de valeur extrême. Plus difficile d'évaluer la pertinence sur cet exemple. La taille de la séquence (5) doit correspondre à un cycle intéressant.
## 2.5 Forêt d'isolement
#### À utiliser pour les anomalies collectives (non ordonnées).
Simple, fonctionne bien avec différentes répartitions de données et efficace avec des données de grande dimension.

In [ ]:
# Prenez des fonctionnalités utiles et normalisez-les
data = df[['value', 'hours', 'daylight', 'DayOfTheWeek', 'WeekDay']]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# train isolement forêt
model =  IsolationForest(contamination = outliers_fraction)
model.fit(data)
# ajouter les données à la main
df['anomaly25'] = pd.Series(model.predict(data))
df['anomaly25'] = df['anomaly25'].map( {1: 0, -1: 1} )
print(df['anomaly25'].value_counts())

In [ ]:
# visualisation de l'anomalie dans le temps (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly25'] == 1, ['time_epoch', 'value']] #anomaly

ax.plot(df['time_epoch'], df['value'], color='blue')
ax.scatter(a['time_epoch'],a['value'], color='red')
plt.show()

In [ ]:
# visualisation d'anomalie avec répartition de température (viz 2)
a = df.loc[df['anomaly25'] == 0, 'value']
b = df.loc[df['anomaly25'] == 1, 'value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label = ['normal', 'anomaly'])
plt.legend()
plt.show()

## 2.6 Une classe SVM
#### A utiliser pour les anomalies collectives (non ordonnées).
Bon pour la détection de nouveauté (pas d'anomalies dans la rame). Cet algorithme fonctionne bien pour les données multimodales.

In [ ]:
# Prenez des fonctionnalités utiles et normalisez-les
data = df[['value', 'hours', 'daylight', 'DayOfTheWeek', 'WeekDay']]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data)
# former une classe SVM
model =  OneClassSVM(nu=0.95 * outliers_fraction) #nu=0.95 * outliers_fraction  + 0.05
data = pd.DataFrame(np_scaled)
model.fit(data)
# ajouter les données à la main
df['anomaly26'] = pd.Series(model.predict(data))
df['anomaly26'] = df['anomaly26'].map( {1: 0, -1: 1} )
print(df['anomaly26'].value_counts())

In [ ]:
# visualisation de l'anomalie dans le temps (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly26'] == 1, ['time_epoch', 'value']] #anomaly

ax.plot(df['time_epoch'], df['value'], color='blue')
ax.scatter(a['time_epoch'],a['value'], color='red')
plt.show()

In [ ]:
# visualisation d'anomalie avec répartition de température (viz 2)
a = df.loc[df['anomaly26'] == 0, 'value']
b = df.loc[df['anomaly26'] == 1, 'value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'], label=['normal', 'anomaly'])
plt.legend()
plt.show()

Donne un résultat similaire à la forêt d'isolement mais trouve quelques anomalies dans les valeurs moyennes. Difficile de savoir si c'est pertinent.
## 2.7 RNN
#### Utiliser pour les anomalies séquentielles (ordonnées)
RNN apprend à reconnaître la séquence dans les données, puis à faire une prédiction basée sur la séquence précédente. Nous considérons une anomalie lorsque les prochains points de données sont éloignés de la prédiction RNN. L'agrégation, la taille de la séquence et la taille de la prédiction de l'anomalie sont des paramètres importants pour avoir une détection pertinente.
Ici on fait apprendre à partir de 50 valeurs précédentes, et on prédit juste la 1 valeur suivante.

In [ ]:
#sélectionner et standardiser les données
data_n = df[['value', 'hours', 'daylight', 'DayOfTheWeek', 'WeekDay']]
min_max_scaler = preprocessing.StandardScaler()
np_scaled = min_max_scaler.fit_transform(data_n)
data_n = pd.DataFrame(np_scaled)

# paramètres importants et taille du train/test
prediction_time = 1 
testdatasize = 1000
unroll_length = 50
testdatacut = testdatasize + unroll_length  + 1

#train data
x_train = data_n[0:-prediction_time-testdatacut].as_matrix()
y_train = data_n[prediction_time:-testdatacut  ][0].as_matrix()

# test data
x_test = data_n[0-testdatacut:-prediction_time].as_matrix()
y_test = data_n[prediction_time-testdatacut:  ][0].as_matrix()

In [ ]:
#unroll: créer une séquence de 50 points de données précédents pour chaque point de données
def unroll(data,sequence_length=24):
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)

# adapter les jeux de données pour la forme des données de séquence
x_train = unroll(x_train,unroll_length)
x_test  = unroll(x_test,unroll_length)
y_train = y_train[-x_train.shape[0]:]
y_test  = y_test[-x_test.shape[0]:]

# voir la forme
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

In [ ]:
# bibliothèques spécifiques pour RNN
# keras est une couche haute construite sur la couche Tensorflow pour rester dans une mise en œuvre de haut niveau/facile
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time #helper libraries
from keras.models import model_from_json
import sys

In [ ]:
# Construire le modèle
model = Sequential()

model.add(LSTM(
    input_dim=x_train.shape[-1],
    output_dim=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    units=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print('compilation time : {}'.format(time.time() - start))

In [ ]:
# Former le modèle
#nb_epoch = 350

model.fit(
    x_train,
    y_train,
    batch_size=3028,
    nb_epoch=30,
    validation_split=0.1)


In [ ]:
# enregistrer le modèle car la formation est longue (1h30) et on ne veut pas le faire à chaque fois
"""
# serialize model to JSON
model_json = model.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model2.h5")
print("Saved model to disk")
"""

In [ ]:
# charger json et créer un modèle
"""
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
"""

In [ ]:
# créer la liste des différences entre les données de prédiction et de test
loaded_model = model
diff=[]
ratio=[]
p = loaded_model.predict(x_test)
# prédictions = lstm.predict_sequences_multiple(loaded_model, x_test, 50, 50)
for u in range(len(y_test)):
    pr = p[u][0]
    ratio.append((y_test[u]/pr)-1)
    diff.append(abs(y_test[u]- pr))

In [ ]:
# tracer la prédiction et la réalité (pour les données de test)
fig, axs = plt.subplots()
axs.plot(p,color='red', label='prediction')
axs.plot(y_test,color='blue', label='y_test')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# sélectionner les points de données de prédiction/réalité les plus éloignés comme anomalies
diff = pd.Series(diff)
number_of_outliers = int(outliers_fraction*len(diff))
threshold = diff.nlargest(number_of_outliers).min()
# données avec étiquette d'anomalie (partie des données de test)
test = (diff >= threshold).astype(int)
# la partie données d'entraînement où on n'a rien prédit (surajustement possible) : pas d'anomalie
complement = pd.Series(0, index=np.arange(len(data_n)-testdatasize))
# # ajouter les données à la main
df['anomaly27'] = complement.append(test, ignore_index='True')
print(df['anomaly27'].value_counts())

In [ ]:
# visualisation de l'anomalie dans le temps (viz 1)
fig, ax = plt.subplots()

a = df.loc[df['anomaly27'] == 1, ['time_epoch', 'value']] #anomaly

ax.plot(df['time_epoch'], df['value'], color='blue')
ax.scatter(a['time_epoch'],a['value'], color='red')
plt.axis([1.370*1e7, 1.405*1e7, 15,30])
plt.show()

In [ ]:
# visualisation d'anomalie avec répartition de température (viz 2)
a = df.loc[df['anomaly27'] == 0, 'value']
b = df.loc[df['anomaly27'] == 1, 'value']

fig, axs = plt.subplots()
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'])
plt.legend()
plt.show()

## Anomalies collectives et séquentielles (Ordonnées)
Cette classe est la plus générale et considère l'ordre ainsi que les combinaisons de valeurs. Nous utilisons généralement une combinaison d'algorithmes comme le modèle cluster + markov.
## 3 Comparaison des résultats
(peut-être plus tard)
## 4 Conclusion
Dans ce cas, la détection d'anomalies contextuelles (catégories+enveloppe elliptique) semble une bonne solution.
